# Import Library

In [7]:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup

# Extract Data

In [23]:
# From csv
electricity_access_precent = pd.read_csv('DataSource/electricity_access_precent.csv', quotechar='"')

rural_population_percent = pd.read_csv("DataSource/rural_population_percent.csv", quotechar='"')

gdp_data = pd.read_csv('DataSource/gdp_data.csv', quotechar='"')

mystery = pd.read_csv('DataSource/mystery.csv', quotechar='"')

population_data_from_csv = pd.read_csv('DataSource/population_data.csv', quotechar='"')

projects_data = pd.read_csv("DataSource/projects_data.csv", quotechar='"')


# From json
population_data_from_json = pd.read_json('DataSource/population_data.json')


# From db
conn = sqlite3.connect('DataSource/population_data.db')
population_data_from_db = pd.read_sql_query("SELECT * FROM population_data;", conn)
conn.close()


# From xml
with open("DataSource/population_data.xml", "r") as f:
    xml_data = f.read()

soup = BeautifulSoup(xml_data, "xml")
data = []
for record in soup.find_all("record"):
    record_data = {}
    for field in record.find_all("field"):
        name = field.get("name")
        value = field.text
        record_data[name] = value
    data.append(record_data)

population_data_from_xml = pd.DataFrame(data)


# from API
url = f"https://api.worldbank.org/v2/countries/all/indicators/SP.POP.TOTL/?format=json&per_page=1000"
data = []
page = 1
total_pages = None

while total_pages is None or page <= total_pages:
    response = requests.get(url, params={"page": page})
    response_data = response.json()
    if total_pages is None:
        total_pages = response_data[0]['pages']
    data.extend(response_data[1])
    page += 1

df_worldbank = pd.json_normalize(data)

C:\Users\SINTA\AppData\Local\Temp\ipykernel_26280\3782938366.py:12: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  projects_data = pd.read_csv("DataSource/projects_data.csv", quotechar='"')


# Transfrom

#### - Check Missing Value + Drop Duplicate

In [11]:
# Cek missing value part 1
mv_electricity_access_percent = electricity_access_precent.isnull().sum()
mv_rural_population_percent = rural_population_percent.isnull().sum()
mv_gdp_data = gdp_data.isnull().sum()
mv_mystery = mystery.isnull().sum()
mv_population_data_from_csv = population_data_from_csv.isnull().sum()
mv_population_data_from_db = population_data_from_db.isnull().sum()
mv_population_data_from_json = population_data_from_json.isnull().sum()
mv_population_data_from_xml = population_data_from_xml.isnull().sum()
mv_projects_data = projects_data.isnull().sum()
mv_df_worldbank = df_worldbank.isnull().sum()

print(mv_electricity_access_percent)
print("-----")
print(mv_rural_population_percent)
print("-----")
print(mv_gdp_data)
print("-----")
print(mv_mystery)
print("-----")
print(mv_population_data_from_csv)
print("-----")
print(mv_population_data_from_db)
print("-----")
print(mv_population_data_from_json)
print("-----")
print(mv_population_data_from_xml)
print("-----")
print(mv_projects_data)
print("-----")
print(mv_df_worldbank)

Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960              264
                 ... 
2014                3
2015                3
2016                3
2017              264
Unnamed: 62       264
Length: 63, dtype: int64
-----
Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960                5
                 ... 
2014                4
2015                4
2016                4
2017                4
Unnamed: 62       264
Length: 63, dtype: int64
-----
Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960              140
                 ... 
2014               17
2015               18
2016               23
2017               30
Unnamed: 62       264
Length: 63, dtype: int64
-----
Unnamed: 0        0
Country Name      0
Country Code      0
Indicator Name    0
Indicator Code    0
                 ..
2013              2
2014              2
2015              2
2

In [32]:
# Cek missing value part 2

## population data
index_indicator_code = population_data_from_csv.columns.get_loc('Indicator Code')
column_after_index = population_data_from_csv.columns[index_indicator_code + 1:]
empty_row = population_data_from_csv[population_data_from_csv[column_after_index].isnull().all(axis=1)]
print(empty_row)
population_data_from_csv = population_data_from_csv.drop(empty_row.index)
print("------")

index_indicator_code = population_data_from_db.columns.get_loc('Indicator_Code')
column_after_index = population_data_from_db.columns[index_indicator_code + 1:]
empty_row = population_data_from_db[population_data_from_db[column_after_index].isnull().all(axis=1)]
print(empty_row)
population_data_from_csv = population_data_from_db.drop(empty_row.index)
print("------")

index_indicator_code = population_data_from_json.columns.get_loc('Indicator Code')
column_after_index = population_data_from_json.columns[index_indicator_code + 1:]
empty_row = population_data_from_json[population_data_from_json[column_after_index].isnull().all(axis=1)]
print(empty_row)
population_data_from_csv = population_data_from_json.drop(empty_row.index)
print("------")

index_indicator_code = population_data_from_xml.columns.get_loc('Indicator Code')
column_after_index = population_data_from_xml.columns[index_indicator_code + 1:]
empty_row = population_data_from_xml[population_data_from_xml[column_after_index].isnull().all(axis=1)]
print(empty_row)
population_data_from_csv = population_data_from_xml.drop(empty_row.index)
print("------")

Empty DataFrame
Columns: [Country Name, Country Code, Indicator Name, Indicator Code, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, Unnamed: 62]
Index: []

[0 rows x 63 columns]
------
     index    Country_Name Country_Code     Indicator_Name Indicator_Code  \
108    108  Not classified          INX  Population, total    SP.POP.TOTL   

     1960  1961  1962  1963  1964  ...  2008  2009  2010  2011  2012  2013  \
108   NaN   NaN   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   

     2014  2015  2016  2017  
108   NaN   NaN   NaN   NaN  

[1 rows x 63 columns]
------
       Country Name Country Code     Indicator Name Indicator Code  1960  \
108  Not classified          INX  P

KeyError: 'Indicator Code'

In [34]:
# Drop duplicate
# dpl_electricity_access_precent = electricity_access_precent.duplicated()
# df = pd.DataFrame(dpl_electricity_access_precent)
# df.to_csv("tes.csv")


# rural_population_percent
# projects_data
# df_worldbank
# mystery
# gdp_data
# population_data_from_csv
# population_data_from_db
# population_data_from_json
population_data_from_xml.to_csv("tes2.csv")

#### - Cleaning data electricity_access_percent & rural_population_percent

In [3]:
electricity_access_precent.drop(columns=["Unnamed: 62"], inplace=True)
electricity_access_precent = electricity_access_precent.fillna(0.000)
electricity_access_precent.drop(columns=["Indicator Name","Indicator Code"], inplace=True)
electricity_access_precent = electricity_access_precent.rename(columns={'Country Name':'country_name','Country Code':'country_code'})
electricity_access_precent = electricity_access_precent.melt(id_vars=["country_name","country_code"], var_name='year', value_name='electricity_access_percent')

rural_population_percent.drop(columns=["Unnamed: 62"], inplace=True)
rural_population_percent = rural_population_percent.fillna(0.000)
rural_population_percent.drop(columns=["Indicator Name","Indicator Code"], inplace=True)
rural_population_percent = rural_population_percent.rename(columns={'Country Name':'country_name','Country Code':'country_code'})
rural_population_percent = rural_population_percent.melt(id_vars=["country_name","country_code"], var_name='year', value_name='rural_population_percent')

#### - Merge electricity_access_percent & rural_population_percent

In [4]:
combine_electricity_rural = pd.merge(electricity_access_precent, rural_population_percent, how='left', on=["country_name", "country_code", "year"])

#### - Cleaning all data population (csv,json,db,xml)

In [6]:
# population_data_from_db.head()
population_data_from_xml.head()

,Country or Area,Item,Year,Value
0,Aruba,"Population, total",1960,54211
1,Aruba,"Population, total",1961,55438
2,Aruba,"Population, total",1962,56225
3,Aruba,"Population, total",1963,56695
4,Aruba,"Population, total",1964,57032


In [53]:
# electricity_access_precent.to_csv("tes2.cvs")
# rural_population_percent
# electricity_access_precent
# print(electricity_access_precent.dtypes)
# print(rural_population_percent.dtypes)
# combine

# combine2 = pd.merge(electricity_access_precent, rural_population_percent, how='left', on=["country_name", "country_code", "year"])
# combine2.to_csv('combine2.csv')

In [3]:
# Handle missing value
# Handle duplikat
# Merge data rural_population_percent + electricity_access_percent